<a href="https://colab.research.google.com/github/siddharth1704/Twitter_sentiment/blob/main/finalsentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import os

In [7]:
import numpy as np
from numpy import array

In [8]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving file2.csv to file2.csv


In [9]:

import pandas as pd 
import io 
  
df = pd.read_csv(io.BytesIO(uploaded['file2.csv'])) 
print(df)


     Unnamed: 0                                             Tweets  sentiment
0             0  RT @AlistairHaimes: Of the whole omnishambles,...          1
1             1  RT @SteveBakerHW: The case for a new coronavir...          0
2             2  RT @atrupar: "If we tested half, cases would b...          1
3             3  @Jim_Jordan Wear a mask! 224,000+ American sou...          0
4             4  RT @BNODesk: The El Paso-Juárez area on the U....          0
..          ...                                                ...        ...
194         194  RT @DonaldJTrumpJr: Muslim Group Says Cuomo Is...          0
195         195  RT @HKLiberty_Team: #HKSARG relaxes max partic...          1
196         196  RT @AmyMcGrathKY: Sen. McConnell blocked $1,20...          1
197         197  RT @SteveBakerHW: The case for a new coronavir...          1
198         198  RT @nytimes: More than 75,000 coronavirus case...          1

[199 rows x 3 columns]


In [10]:
import re
#cleaning the data
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9,.'']+','',text) #remove @mentions
    text=re.sub(r'#','',text) #Removing the # symbol
    text=re.sub(r'https?:\/\/\S+','',text)#remove hyperlink
    text=re.sub(r'RT[\s]+','',text)#removing RT
    
    return text
#cleaning the new data
df['Tweets']=df['Tweets'].apply(cleanTxt)

In [11]:

import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS
import string


In [12]:
stop_words = list(STOP_WORDS)
pun = string.punctuation

In [13]:
#removing the stopwords
def data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
        
    clean_tokens = []
    for token in tokens:
        if token not in stop_words and token not in pun:
            clean_tokens.append(token)
    return clean_tokens
# Function to convert 
def listToString(s): 
    # initialize an empty string 
    str1 = " "        # return string 
    return (str1.join(s))

In [14]:
ctweets = []
tweets = list(df['Tweets'])
for Tweet in tweets:
    ctweets.append(listToString(data_cleaning(Tweet)))

In [15]:
ctweets[3]

'jordan wear mask 224,000 american soul lose covid19 true patriot wear mask th …'

In [324]:
sentiments = df['sentiment']

from numpy import array
sentiments = np.array(list(sentiments))

In [325]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
ctweets_train, ctweets_test, sentiments_train, sentiments_test = train_test_split(ctweets, sentiments, test_size = 0.20, random_state = 42)

In [326]:

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(ctweets_train)

ctweets_train = tokenizer.texts_to_sequences(ctweets_train)
ctweets_test = tokenizer.texts_to_sequences(ctweets_test)

In [327]:

vocab_size = len(tokenizer.word_index) + 1

maxlen = 5000

ctweets_train = pad_sequences(ctweets_train, padding='post', maxlen = maxlen)
ctweets_test = pad_sequences(ctweets_test, padding='post', maxlen = maxlen)


In [328]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

glove.6B.zip.1       59%[==========>         ] 487.38M  2.23MB/s    eta 2m 35s ^C


In [329]:
!unzip glove*.zip

In [22]:
!ls
!pwd

file2.csv	   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
/content


In [339]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dict = dict()
glove = open('glove.6B.100d.txt', encoding="utf8")

for line in glove:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:],dtype = 'float32')
    embeddings_dict [word] = vector_dimensions
glove.close()

In [341]:
embedding_matrix = zeros((vocab_size,100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [342]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate,LSTM,Conv1D
from keras.layers import  Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
#model = Sequential()
#embedding_layer = Embedding(vocab_size, 100, weights = [embedding_matrix], input_length = maxlen, trainable = False)
#model.add(embedding_layer)
#model.add(SpatialDropout1D(0.3))
#model.add(Flatten())
#model.add(Activation('softmax'))
#model.add(Dense(6, activation = 'relu'))
#model.add(Dense(1, activation = 'sigmoid'))
#model.add(layers.Dense(64, activation='relu'))
#model.add(tf.keras.layers.Dense(16, kernel_initializer='lecun_normal',activation='selu'))



In [333]:

#optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
#model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['acc'])
#print(model.summary())

In [343]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100),
   
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_88"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_88 (Embedding)     (None, None, 100)         67800     
_________________________________________________________________
global_max_pooling1d_28 (Glo (None, 100)               0         
_________________________________________________________________
dense_138 (Dense)            (None, 24)                2424      
_________________________________________________________________
dense_139 (Dense)            (None, 1)                 25        
Total params: 70,249
Trainable params: 70,249
Non-trainable params: 0
_________________________________________________________________


In [344]:
history = model.fit(ctweets_test, sentiments_test, epochs=20, validation_data=(ctweets_test, sentiments_test), verbose=1)

Epoch 1/20
2/2 [==============================] - 0s 87ms/step - loss: 0.6913 - accuracy: 0.6000 - val_loss: 0.6848 - val_accuracy: 0.6000
Epoch 2/20
2/2 [==============================] - 0s 39ms/step - loss: 0.6845 - accuracy: 0.6000 - val_loss: 0.6790 - val_accuracy: 0.6000
Epoch 3/20
2/2 [==============================] - 0s 36ms/step - loss: 0.6783 - accuracy: 0.6000 - val_loss: 0.6729 - val_accuracy: 0.6000
Epoch 4/20
2/2 [==============================] - 0s 36ms/step - loss: 0.6722 - accuracy: 0.6000 - val_loss: 0.6668 - val_accuracy: 0.6000
Epoch 5/20
2/2 [==============================] - 0s 35ms/step - loss: 0.6664 - accuracy: 0.6000 - val_loss: 0.6606 - val_accuracy: 0.6000
Epoch 6/20
2/2 [==============================] - 0s 36ms/step - loss: 0.6600 - accuracy: 0.6000 - val_loss: 0.6542 - val_accuracy: 0.6000
Epoch 7/20
2/2 [==============================] - 0s 38ms/step - loss: 0.6534 - accuracy: 0.6000 - val_loss: 0.6478 - val_accuracy: 0.6000
Epoch 8/20
2/2 [===========